In [1]:
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI

load_dotenv()
model = ChatGoogleGenerativeAI(model="gemini-1.5-flash")

/home/prawater/.local/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel, Field
from langchain_core.messages import HumanMessage
import base64

class FoodIdentification(BaseModel):
    name: str = Field(description="Name of the food item")
    weight: float = Field(description="Estimated weight of the food item in grams")

class CalorieAnalysis(BaseModel):
    calories: float = Field(description="Calorie content of the food sample in kcal")
    proteins: float = Field(description="Protein content of the food sample in g")
    fats: float = Field(description="Fats content of the food sample in g")
    carbohydrates: float = Field(description="Carbohydrate content of the food sample in g")
    dietary_fiber: float = Field(description="Fiber content of the food sample in g")
    sugars: float = Field(description="Sugar content of the food sample in g")
    sodium: float = Field(description="Sodium content of the food sample in mg")
    cholesterol: float = Field(description="Cholesterol content of the food sample in mg")

food_parser = JsonOutputParser(pydantic_object=FoodIdentification)
calorie_parser = JsonOutputParser(pydantic_object=CalorieAnalysis)

image_file = "./badam.jpg"
with open(image_file, "rb") as image_file:
    image_data = image_file.read()
base64_image = base64.b64encode(image_data).decode('utf-8')

image_url = f"data:image/jpeg;base64,{base64_image}"


identification_prompt = HumanMessage(content=[
    {"type": "image_url", "image_url": image_url},
    {
        "type": "text",
        "text": f"Step 1: Identify the food item in this image. Step 2: Estimate the weight of the food item in grams. {food_parser.get_format_instructions()}"
    }
])

response_1 = model.invoke([identification_prompt])
parsed_response_1 = food_parser.parse(response_1.content)

print("First API Response:")
print(parsed_response_1)

food_name = parsed_response_1['name']
weight = parsed_response_1['weight']

analysis_prompt = HumanMessage(content=[
    {
        "type": "text",
        "text": f"You have identified the food item as '{food_name}' weighing approximately {weight} grams. Provide a detailed nutritional analysis, including calories, proteins, fats, carbohydrates, dietary fiber, sugars, sodium, and cholesterol. {calorie_parser.get_format_instructions()}"
    }
])

response_2 = model.invoke([analysis_prompt])
parsed_response_2 = calorie_parser.parse(response_2.content)

print("Second API Response:")
print(parsed_response_2)

First API Response:
{'name': 'Almonds', 'weight': 50}
Second API Response:
{'calories': 165, 'proteins': 6, 'fats': 14, 'carbohydrates': 6, 'dietary_fiber': 3.5, 'sugars': 1, 'sodium': 1, 'cholesterol': 0}
